In [8]:
#import all the required
import datetime
import numpy as np
import os
import pandas as pd
import time
import re
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

In [ ]:
driver=webdriver.Chrome()  # open the chrome 
driver.maximize_window()   # maximize the window
driver.get("https://www.zomato.com/bangalore")  #to the web page

In [18]:
# dictionary  of required column
data = {"name":[],
              "price":[],
              "cuisine":[],
              "location":[],
              "link":[],
              "rating":[]}

In [22]:
#Looping over to get the data and append them in the dictionary 
for i in range(5):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # get the screen height of the web
    time.sleep(5)
    content = driver.find_elements(By.XPATH,"//div[@class='jumbo-tracker']") # parent class of each element 

    for j in content:
        try:
            data["name"].append(j.find_element(By.CLASS_NAME, "sc-1hp8d8a-0.sc-bpubUI.eKeNGz").text)
        except:
            data["name"].append(None)
        try:
            data["price"].append(j.find_element(By.CLASS_NAME, "sc-1hez2tp-0.sc-sVRsr.jIrGje").text)
        except:
            data["price"].append(None)
        try:
            data["cuisine"].append(j.find_element(By.CLASS_NAME, "sc-1hez2tp-0.sc-sVRsr.gVSfmH").text)
        except:
            data["cuisine"].append(None)
        try:
            data["location"].append(j.find_element(By.CLASS_NAME, "sc-1hez2tp-0.sc-jkPxnQ.jKgMiA").text)
        except:
            data["location"].append(None)
        try:
            data["link"].append(j.find_element(By.CLASS_NAME, "sc-btewqU.dlikcC").get_attribute("href"))
        except:
            data["link"].append(None)
        try:
            data["rating"].append(j.find_element(By.CLASS_NAME, "sc-1q7bklc-1.cILgox").text)
        except:
            data["rating"].append(None)

    time.sleep(3)

driver.quit()    # close the webpage

In [ ]:
for i in data:            # check length of each key  
    print(len(data[i]))   

In [ ]:
# Find the length of the longest list
max_len = max(len(v) for v in data.values())

# Fill shorter lists with None or any other placeholder value
for key in data:
    if len(data[key]) < max_len:
        data[key].extend([None] * (max_len - len(data[key])))

In [ ]:
zomato_first_page = pd.DataFrame(data)    # convert into dataframe

In [ ]:
zomato_first_page 

In [ ]:
zomato_first_page.info()    # know the complete information about the dataframe

In [ ]:
#cleaning the data (price)
zomato_first_page["price"] = zomato_first_page["price"].str.replace(r'\D', '', regex=True).astype(int)
zomato_first_page["price"]

In [ ]:
#(replacing the New with 0)
zomato_first_page.loc[zomato_first_page['rating'] == 'New', 'rating'] = 0

In [ ]:
for i in zomato_first_page:                         # checking the length  of each column.
    print(len(zomato_first_page[i]))  

In [ ]:
zomato_first_page.to_csv("zomato_first_page.csv")

In [ ]:


                              #SCRAPING OF SECOND PAGE STARTS#




In [30]:
           
second_page={"name":[] ,"Timings":[],"Resturant_known_for":[], "delivery_review_number": [], "dish_name": []} 

In [38]:

driver = webdriver.Chrome() #OPEN CHROME
driver.maximize_window() # MAXIMIZE WINDOW


#for link in data["link"]:
for url in data["link"]:
    driver.get(url) #OPEN URL 
    try:
        name_element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".sc-7kepeu-0.sc-iSDuPN.fwzNdh")) # WAIT UNTIL PRESENCE LOCATED
        )
        N = name_element.text if name_element else None    # GET THE TEXT IF FOUND ELSE NONE
    except:
        N = None
    time.sleep(4)    # SLEEP TO AVOID REJECTION FROM WEBSITE
    try:
        rating=driver.find_element(By.CSS_SELECTOR,".sc-dxZgTM.hOUWLw")# TARGET PARENT CLASSES
        delivery=rating.find_elements(By.CSS_SELECTOR,".sc-1q7bklc-8.kEgyiI") # TARGET SIBLING CLASSES
        d=delivery[1].text  # GET THE ELEMENT AT INDEX 1
    except:
        d= None

    try:            #SINCE THE CLASS OF DISH CHANGES FIND THE ANCHOR TAG WHICH HAS TEXT AS MENTIONOED AND IN THAT FIND THE SIBLING CLASS P WHICH HOLDS DISHES TEXT
        popular_element = WebDriverWait(driver, 4).until(
            EC.presence_of_element_located((By.XPATH, "//h3[text()='Popular Dishes']/following-sibling::p"))
        )
        dish = popular_element.text if popular_element else None  #GET THE TEXT IF FOUND ELSE NONE
    except:
        dish = None
    time.sleep(4)    # SLEEP TO AVOID REJECTION FROM WEBSITE
    try:           
        second_page["Timings"].append(driver.find_element(By.XPATH, "/html/body/div[1]/div/main/div/section[3]/section/section/div/div/section[2]/section/span[2]").text)
    except Exception as e:
        if '404' in str(e):
            second_page["Timings"].append(None)  # Append None if 404 error occurs
        else:
            second_page["Timings"].append(None)  # Append None for any other errors
        continue  # Continue with the next URL
  
    try:                  #SINCE THE CLASS OF DISH CHANGES FIND THE ANCHOR TAG WHICH HAS TEXT AS MENTIONOED AND IN THAT FIND THE SIBLING CLASS P WHICH HOLDS DISHES TEXT
        second_page["Resturant_known_for"].append(driver.find_element(By.XPATH, "/html/body/div[1]/div/main/div/section[4]/section/section/article[1]/section[1]/p[2]").text)   
    except:
        second_page["Resturant_known_for"].append(None)
    

    second_page["name"].append(N)                      #ADDING THE VALUES EXTRACTED
    second_page["delivery_review_number"].append(d)
    second_page["dish_name"].append(dish)
    #second_page["Timings"].append(time)

driver.quit()  # Close the browser after the loop




In [44]:
# Find the length of the longest list
max_len = max(len(v) for v in second_page.values())

# Fill shorter lists with None or any other placeholder value
for key in second_page:
    if len(second_page[key]) < max_len:
        second_page[key].extend([None] * (max_len - len(second_page[key])))
        
#created the dataframe out of dictionary
zomato_second_page= pd.DataFrame(second_page)
#view the dataframe with its values
zomato_second_page.head()

,name,Timings,Resturant_known_for,delivery_review_number,dish_name
0,Muro,"12midnight – 1am, 12noon – 11:45pm (Today)","Exclusive of applicable taxes and charges, if any",0,None
1,Flo,12noon – 1am (Today),"Amazing Food at Amazing Price, Economical Pric...",28,"Cocktails, Drinks, Beer, Desserts, Starters"
2,Secret Story,12noon – 1am (Today),"Staff Behaviour is Excellent, Best in Service,...",0,"Vietnamese Summer Roll, Sundried Tomato Pizza,..."
3,Oney Brewhouse,12noon – 12midnight (Today),"Exclusive of applicable taxes and charges, if any",0,None
4,Scottish Pub,12noon – 12midnight (Today),"Great Place, Nice Place, Prices, Good Place, S...",0,"Cheese Cherry Pineapple, Chicken Pakoda, Wine,..."


In [48]:
zomato_second_page.tail(10)   #CHECK DATA

,name,Timings,Resturant_known_for,delivery_review_number,dish_name
485,The Pizza Bakery,Opens at 7pm,"Service is Great, Clean and Hygienic Place, Sa...","2,142","Garlic Bread Basket, Tiramisu, Onion Rings, Hu..."
486,SodaBottleOpenerWala,11am – 11:30pm (Today),"Rasberry Soda, Chicken Sanju Baba, Bohri Keema...","1,847","Rasberry Soda, Chicken Sanju Baba, Bohri Keema..."
487,Thai Basil,10am – 10:30pm (Today),"Cozy Ambience, Gastronomical Experience, Good ...",713,"Tom Yum Prawn, Thai Spice, Sliced Beef, Mango ..."
488,Masaledaar Desi Rasoi,11am – 12midnight (Today),₹800 for two people (approx.),417,None
489,Full Circle,11:30am – 11:30pm (Today),"Reasonably Priced Food, Cool Ambience, Service...",472,"Cheesecake, Cocktails, Drinks, Pasta, Chicken"
490,Hoy Punjab Restaurant,12noon – 11pm (Today),"Exclusive of applicable taxes and charges, if any",6,None
491,None,11:30am – 10:50pm (Today),None,None,None
492,None,Opens at 7pm,None,None,None
493,None,12noon – 1am (Today),None,None,None
494,None,11am – 11pm (Today),None,None,None


In [50]:
zomato_second_page.to_csv("zomato_second_page.csv") # CONVERT EXTRACTED DATA TO CSV FILE 